# Поиск оптимальной марковской цепи из 2 состояний (вероятности перехода p и q) для последовательности из 0 и 1

p и q - вероятности, числа в интервале (0, 1). $C(\mu) = $ # знаков после запятой

In [1]:
# хочется уметь переводить десятичные дроби в двоичные
def binFloat(n,k): # n - число, k - точность двоичной дроби, которую хотим
    s = bin(int(n))[2:] + '.'# перевели целую часть в двоичную систему
    r = n - int(n) # дробная часть n
    while len(s) < k and r > 0:
        #print(r)
        r *= 2
        if r >= 1:
            s += '1'
            r -= 1
        else:
            s += '0'
    return s

In [2]:
# пример работы
binFloat(0.125,10)

'0.001'

In [3]:
# наоборот - дробное двоичное число в десятичную дробь
def floatBin(s):
    if '.' not in s:
        return int(s,2)

    a,b = s.split('.')
    res = int(a,2)
    for i in range(1,len(b)+1):
        res += int(b[i-1]) * 2**(-i)
    return res

In [4]:
# пример работы
from math import pi
print(round(pi, 10))
print(binFloat(pi,30))

3.1415926536
11.001001000011111101101010100


In [5]:
print(floatBin('11.0010010000111'))
print('почти совпало и пи')

3.1414794921875
почти совпало и пи


In [6]:
from pdb import Restart
from math import log2
from math import sqrt
from itertools import product
import time

def logProb(s,p,q): # возвращает log2 вероятности получить строку s с переходными вероятностями p из 0 в 1 И q из 1 в 0
    s = s.replace('.','')
    # считаем переходы между состояниями 00, 01, 10, 11
    n00 = s.count('00')
    n01 = s.count('01')
    n10 = s.count('10')
    n11 = s.count('11')
    # тут нужны допущения на log2 при 0 и 1, чтобы python не ругался на математику при подсчёте log2(0)
    if p == 0:
        log_p = -1e10 
        log_1_p = 0
    elif p == 1:
        log_1_p = -1e10 
        log_p = 0
    else:
        log_p = log2(p)
        log_1_p = log2(1-p)
    if q == 0:
        log_q = -1e10 
        log_1_q = 0
    elif q == 1:
        log_1_q = -1e10 
        log_q = 0
    else:
        log_q = log2(q)
        log_1_q = log2(1-q)
    log_q = -1e10 if q == 0 else log2(q)

    # собственно, вероятность получить строку s
    res = n01*log_p + n10*log_q + n00*log_1_p + n11*log_1_q
    return res

# v 1.1

def MDL(s,k,l):
    m = 10**30 # минимум пусть сначала будет таким большим, чтобы было, с чем сравнивать
    # генерируем все возможные двоичные числа в [0, 1] с k и l знаков после запятой
    p_range = ['0.'+''.join(i) for i in product('01',repeat=k)] + ['1.']
    q_range = ['0.'+''.join(i) for i in product('01',repeat=l)] + ['1.']
    for p in p_range:
        for q in q_range:
            lp = len(p)-1
            lq = len(p)-1
            pf = floatBin(p)
            qf = floatBin(q)
            r = lp+lq-logProb(s,pf,qf) # Rissanen complexity
            if r < m: # условие обновления минимума и оптимальных p и q
                m = r
                p0 = pf
                q0 = qf
    return (m, p0, q0) # выводит minimun description length, p, q
    

In [ ]:
# генерируем все последовательности из 0 и 1 длины k - предполагаемые вероятности
from itertools import product
k = 10
s_product = product('01',repeat=k)
s = ['0.'+''.join(i) for i in s_product] + ['1.']
print(s)
f = [floatBin(i) for i in s]
print(f)

['0.0000000000', '0.0000000001', '0.0000000010', '0.0000000011', '0.0000000100', '0.0000000101', '0.0000000110', '0.0000000111', '0.0000001000', '0.0000001001', '0.0000001010', '0.0000001011', '0.0000001100', '0.0000001101', '0.0000001110', '0.0000001111', '0.0000010000', '0.0000010001', '0.0000010010', '0.0000010011', '0.0000010100', '0.0000010101', '0.0000010110', '0.0000010111', '0.0000011000', '0.0000011001', '0.0000011010', '0.0000011011', '0.0000011100', '0.0000011101', '0.0000011110', '0.0000011111', '0.0000100000', '0.0000100001', '0.0000100010', '0.0000100011', '0.0000100100', '0.0000100101', '0.0000100110', '0.0000100111', '0.0000101000', '0.0000101001', '0.0000101010', '0.0000101011', '0.0000101100', '0.0000101101', '0.0000101110', '0.0000101111', '0.0000110000', '0.0000110001', '0.0000110010', '0.0000110011', '0.0000110100', '0.0000110101', '0.0000110110', '0.0000110111', '0.0000111000', '0.0000111001', '0.0000111010', '0.0000111011', '0.0000111100', '0.0000111101', '0.0000

1) 001001...0010 (длина 3001)

In [ ]:
s = '001' * 1000 + '0'
print('Mimimum description lenght, p, q:')
MDL(s,10,10)

Mimimum description lenght, p, q:


(2022.0, 0.5, 1)

Как и ожидалось по аналитическим выкладкам

2) $\pi\approx3,14159265$

In [ ]:
s = binFloat(pi,30)
print(round(pi,30))
print(s)
print('Mimimum description lenght, p, q:')

start = time.time()

MDL(s,10,10)

end = time.time() - start ## время работы программы


'''3.141592653589793
11.001001000011111101101010100
Mimimum description lenght, p, q:
(46.2545056086638, 0.5830078125, 0.615234375)'''
print('Time:',end)

3.141592653589793
11.001001000011111101101010100
Mimimum description lenght, p, q:
Time: 12.053530931472778


In [ ]:
smpl = list(range(5,11))
arr = [[0]*6]*6

start = time.time()

for k in smpl:
    for l in smpl:
        arr[k-5][l-5] = MDL(s,k,l)

end = time.time() - start ## время работы программы
for i in arr:
    for j in i:
        print(j, end=' & ')
    print('\\\\')

'''3.141592653589793
11.001001000011111101101010100
Mimimum description lenght, p, q:
(46.2545056086638, 0.5830078125, 0.615234375)'''
print('Time:',end)

(46.25819159033924, 0.5830078125, 0.625) & (46.2559301532723, 0.5830078125, 0.609375) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.2545056086638, 0.5830078125, 0.615234375) & (46.2545056086638, 0.5830078125, 0.615234375) & \\
(46.25819159033924, 0.5830078125, 0.625) & (46.2559301532723, 0.5830078125, 0.609375) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.2545056086638, 0.5830078125, 0.615234375) & (46.2545056086638, 0.5830078125, 0.615234375) & \\
(46.25819159033924, 0.5830078125, 0.625) & (46.2559301532723, 0.5830078125, 0.609375) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.2545056086638, 0.5830078125, 0.615234375) & (46.2545056086638, 0.5830078125, 0.615234375) & \\
(46.25819159033924, 0.5830078125, 0.625) & (46.2559301532723, 0.5830078125, 0.609375) & (46.25463364705105, 0.5830078125, 0.6171875) & (46.25463364705105, 0.

In [ ]:
def tablePrint(arr):
    print('\\begin{tabular}{|' + 'l|' * (len(smpl) + 1) + '}')
    print('\\hline')
    print('k / l &' + ' & '.join([str(i) for i in smpl]) + '\\\\')
    ind = 0
    print('\\hline')
    for i in arr:
        print(smpl[ind],end = ' ')
        for k in range(3):
            for j in i:
                print('&', round(j[k],4), end='')
            print('\\\\')
        print('\\hline')
        ind += 1
    print('\\end{tabular}')

tablePrint(arr)

\begin{tabular}{|l|l|l|l|l|l|l|}
\hline
k / l &5 & 6 & 7 & 8 & 9 & 10\\
\hline
5 & 46.2582& 46.2559& 46.2546& 46.2546& 46.2545& 46.2545\\
& 0.583& 0.583& 0.583& 0.583& 0.583& 0.583\\
& 0.625& 0.6094& 0.6172& 0.6172& 0.6152& 0.6152\\
\hline
6 & 46.2582& 46.2559& 46.2546& 46.2546& 46.2545& 46.2545\\
& 0.583& 0.583& 0.583& 0.583& 0.583& 0.583\\
& 0.625& 0.6094& 0.6172& 0.6172& 0.6152& 0.6152\\
\hline
7 & 46.2582& 46.2559& 46.2546& 46.2546& 46.2545& 46.2545\\
& 0.583& 0.583& 0.583& 0.583& 0.583& 0.583\\
& 0.625& 0.6094& 0.6172& 0.6172& 0.6152& 0.6152\\
\hline
8 & 46.2582& 46.2559& 46.2546& 46.2546& 46.2545& 46.2545\\
& 0.583& 0.583& 0.583& 0.583& 0.583& 0.583\\
& 0.625& 0.6094& 0.6172& 0.6172& 0.6152& 0.6152\\
\hline
9 & 46.2582& 46.2559& 46.2546& 46.2546& 46.2545& 46.2545\\
& 0.583& 0.583& 0.583& 0.583& 0.583& 0.583\\
& 0.625& 0.6094& 0.6172& 0.6172& 0.6152& 0.6152\\
\hline
10 & 46.2582& 46.2559& 46.2546& 46.2546& 46.2545& 46.2545\\
& 0.583& 0.583& 0.583& 0.583& 0.583& 0.583\\
& 0.625& 0.

3) $\sqrt{2}$

In [ ]:
s = binFloat(sqrt(2),30)
print(round(sqrt(2),30))
print(s)
print('Mimimum description lenght, p, q:')
MDL(s,10,10)

1.4142135623730951
1.0110101000001001111001100110
Mimimum description lenght, p, q:


(46.2545056086638, 0.5830078125, 0.615234375)

Получили результат, идетичный числу пи. Возможно, для непериодических дробей есть свое оптимальное значение p и q. Интересно, что золотое сечение — это деление величины в отношении 62 % и 38 %, а мы получаем $q \approx 0.615$

4) $\sqrt{3}$

In [ ]:
s = binFloat(sqrt(3),30)
print(round(sqrt(3),30))
print(s)
print('Mimimum description lenght, p, q:')
MDL(s,10,10)

1.7320508075688772
1.1011101101100111101011101000
Mimimum description lenght, p, q:


(43.82972284464091, 0.77734375, 0.533203125)

Здесь результат уже немного отличается, причём Minimal description length похожая: 46, 43.

# Черновик


In [ ]:
"""
def prob(s,p,q):
    s = s.replace('.','')
    # считаем переходы между состояниями 00, 01, 10, 11
    n00 = s.count('00')
    n01 = s.count('01')
    n10 = s.count('10')
    n11 = s.count('11')
    #print(n00,n01,n10,n11)
    res = p**n01 * q**n10 * (1-p)**n00 * (1-q)**n11
    return res
"""

"""
# v 1.0
def MDL(s,k,l):
    m = 10**30 # minimum
    for i in range(1,2**(k-1)):
        p = i / 2**(k-1)
        for j in range(1,2**(l-1)):
            q = j / 2**(l-1)
            lp = len(binFloat(p,k))-2
            lq = len(binFloat(q,l))-2
            r = lp+lq-logProb(s,p,q) # Rissanen complexity
            if r < m:
                m = r
                p0 = p
                q0 = q
    return (m, p0, q0)
"""